# ChemEnv : a fast and robust tool to automatically identify coordination environments

Written using:
- pymatgen==2019.1.13

This tutorial will show you how to use the *ChemEnv* package to analyze chemical coordination environments and to identify model coordination environments that are closest to the coordination environment at hand. The latter is decided by the calculation of continous symmetry measures (CSM). 

The analysis of the coordination environments proceeds as follows:
1. Search for neighbors by a modified Voronoi analysis on a grid of distance and angular parameters
2. Calculation of corresponding continous symmetry measures (CSMs) with model environments for all distance and angular parameters
3. Refinement of the results with different strategies (e.g., by using certain distance and angular parameters)

## The very first steps

1. Download and install [pymatgen](http://pymatgen.org/)

2. Sign up for the *Materials Project* on [https://materialsproject.org/](https://materialsproject.org/).

3. Setup the connection to the *Materials Project* with in your terminal: 
   `pmg config --add PMG_MAPI_KEY <USER_API_KEY>`

## Import the relevant modules

Let us start by importing the relevant modules.

In [2]:
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
import logging
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy, MultiWeightsChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments

## Load a structure from the materials project

Next, we will load the crystal structure of our interest. We will start with a very simple
example here: 伪-quartz which is depicted below.

In [3]:
# Get a structure from the materials project (alpha-quartz)
# a = MPRester()
M1 = ['Al','Ga','In','Sc']
M2 = ['S','Se']
i = M1[3]
j = M2[0]
struct = Structure.from_file(f"/Users/runix/Desktop/Li4MS4_phonon/LiM_MO3_2/{i}-{j}/relax/CONTCAR")

Here, the structure of alpha-SiO<sub>2</sub> (mp-7000) is depicted with possible coordination environments for Si (blue).


<img src="https://github.com/materialsvirtuallab/matgenb/blob/master/notebooks/Figure_quartz.png?raw=true" width="40%">
The graphic is created with VESTA: K. Momma and F. Izumi, "VESTA 3 for three-dimensional visualization of crystal, volumetric and morphology data," *J. Appl. Cryst.*, **2011**, *44*, 1272.

Alternatively, one can load the structure from a cif-file:

In [4]:
#just remove the comment
#from pymatgen.io.cif import CifParser
#parser = CifParser("mystructure.cif")
#struct = parser.get_structures()[0]
struct

Structure Summary
Lattice
    abc : 9.276165194052474 9.276156845513848 9.276137690845498
 angles : 108.27465903362751 108.27477977555519 111.89193848786307
 volume : 613.6150253325486
      A : -4.134461256574268 -7.50542153609696 -3.5527620791053334
      B : -0.0490248251017412 -0.0890026777615065 9.275600299304
      C : -4.639815149779847 7.505405252750324 -2.8614223434134525
PeriodicSite: Li (-2.1890, -0.9940, 4.1530) [0.3121, 0.6250, 0.1871]
PeriodicSite: Li (-5.5884, 2.8035, -2.7222) [0.4379, 0.1250, 0.8129]
PeriodicSite: Li (-4.7661, 2.2924, 1.9153) [0.3750, 0.5621, 0.6871]
PeriodicSite: Li (-5.1031, -4.2800, 2.3768) [0.8750, 0.6879, 0.3129]
PeriodicSite: Sc (-4.7256, -0.9938, 2.2987) [0.6012, 0.6250, 0.4762]
PeriodicSite: Sc (-3.0518, 2.8035, -0.8681) [0.1488, 0.1250, 0.5238]
PeriodicSite: Sc (-6.1216, 4.4364, 3.7697) [0.3750, 0.8512, 0.9762]
PeriodicSite: Sc (-3.7476, -6.4240, 0.5225) [0.8750, 0.3988, 0.0238]
PeriodicSite: S (-1.9989, 0.8679, 1.6494) [0.1628, 0.3273, 0.2823]

## Setup of the local geometry finder

First, one has to choose a referene point for the Voronoi analysis. The two relevant parameters are `centering_type` and `include_central_site_in_centroid`. `centering_type` can be either `'standard'`, `'centroid'`, or `'central_site'`. The first results in a default setting, the second
one sets the reference point at the centroid of the structure (calculated with or without the
site for which the Voronoi analysis is currently performed depending on the Boolean value of
`include_central_site_in_centroid`). Have a look at the the [code](http://pymatgen.org/_modules/pymatgen/analysis/chemenv/coordination_environments/coordination_geometry_finder.html#LocalGeometryFinder.setup_parameters) for more information. Moreover, a logging is introduced. This is especially important if one runs very large calculations (large distances and small angle parameters) and one wants to access the status of the calculation.

In [5]:
# Setup the local geometry finder
lgf = LocalGeometryFinder()
lgf.setup_parameters(centering_type='centroid', include_central_site_in_centroid=True)

#you can also save the logging to a file, just remove the comment
logging.basicConfig(#filename='chemenv_structure_environments.log',
                    format='%(levelname)s:%(module)s:%(funcName)s:%(message)s',
                    level=logging.DEBUG)
lgf.setup_structure(structure=struct)


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



## Get the StructureEnvironments (=Voronoi analysis plus CSM calculation on a grid of distance and angular parameters)

In the first step, sets of neighbors for different distance and solid angle parameters via Voronoi analysis and corresponding continous symmetry measures (CSMs) are computed. 

Have a close look at [the documentation of the code](http://pymatgen.org/_modules/pymatgen/analysis/chemenv/coordination_environments/coordination_geometry_finder.html#LocalGeometryFinder.compute_structure_environments) for more information on parameters that you can define. For example, we will use the `maximum_distance_factor` to save some computing time. Multiplied with the distance to the closest neighbour it results in the maximum distance that is considered in the Voronoi approach. Similarily, one can define another parameter - the `minimum_angle_factor`. This factor is multiplied with the maximum solid angle to the
neighbours to arrive at the minimum solid angle that is considered in the Voronoi analysis. Don鈥檛 choose to drastic values, please. This might lead to unexpected results. One can also refine these parameters later. For more information on these parameters, please consult the main text of the article again. Moreover, one can also exclude atoms from the computation of the coordination environments (e.g., `se=lgf.compute_structure_environments(excluded_atoms=['O'])`). Additional parameters such as `only_atoms`, `only_indices` and `only_cations` in combination with `valences` might be helpful as well.

In [6]:
# Get the StructureEnvironments
se = lgf.compute_structure_environments(maximum_distance_factor=2.6,excluded_atoms=['S'])


DEBUG:coordination_geometry_finder:compute_structure_environments:Getting DetailedVoronoiContainer
DEBUG:voronoi:__init__:Setting Voronoi list
DEBUG:voronoi:setup_voronoi_list:Getting all neighbors in structure
DEBUG:voronoi:setup_voronoi_list:Setting up Voronoi list :
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #0 (1/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #1 (2/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #2 (3/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #3 (4/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #4 (5/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #5 (6/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #6 (7/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #7 (8/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #8 (9/64)
DEBUG:voronoi:setup_voronoi_list:  - Voronoi analysis for site #9 (10/64)

## Different strategies to analyze the StructureEnvironments

In [7]:
# se.get_csm(4,'T:4')
for i in [0,1,2,3]:
    print(f'{i} sites csm ='+str(se.get_csm(i,'T:4')['symmetry_measure']))

0 sites csm =7.764331474542513
1 sites csm =7.764331474542502
2 sites csm =7.764331474542503
3 sites csm =7.764331474542517


In [8]:
for i in [4,5,6,7]:
    print(f'{i} sites csm ='+str(se.get_csm(i,'O:6')['symmetry_measure']))

4 sites csm =19.39511618087608
5 sites csm =19.39511618087608
6 sites csm =19.39511618087607
7 sites csm =19.39511618087607


Now, the strategy to interpret the data from before is chosen to arrive at information about the coordination environments. One can choose between two different types of strategies: we start with the `SimplestChemenvStrategy`. This strategy type uses fixed angle and distance parameters for the definition of neighbors in the Voronoi approach. The resulting coordination environment is uniquely defined and is then given as the one with the lowest continuous
symmetry measure. One of the disadvantages is that it fails for intermediate coordination environments and depends very much on the cutoff parameters chosen.

Important parameters for this strategy are: `distance_cutoff` and `angle_cutoff`. The strategy is correct in about 85% of the cases if one uses `distance_cutoff=1.4` and `angle_cutoff=0.3`.
The neighbouring atoms that are considered in this approach have a **maximum distance** of `distance_cutoff` 脳 the distance to the closest neighbour and a **minimum solid angle** of `angle_cutoff` 脳 the biggest solid angle. For more information , see [the code](http://pymatgen.org/_modules/pymatgen/analysis/chemenv/coordination_environments/chemenv_strategies.html#SimplestChemenvStrategy).

In [96]:
# strategy = SimplestChemenvStrategy(distance_cutoff=1.42, angle_cutoff=0.3)
# lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)

Next, one can print the information on the coordination environments for each site in the structure. Here, the information for a site occupied by one oxygen is printed. Please be aware that the counting of the sites starts at 0. In case of doubt, please print the respective site of the structure object with print(structure[isite]) where isite is an Integer.

In [97]:
# #print coordination environments for a special site
# isite = 0
# print(se.coordination_environments[isite])

It will return a dictionary including all relevant information. The item corresponding to the key `'ce_symbol'` symbolizes the coordination environment. `'A:2'` represents the angular coordination environment. The full list of coordination environments can be seen in the supporting information of this article. With this strategy, the value of
`'ce_fraction'` is always equal to 1.0. The `'csm'` represents the continous symmetry measure. This value lies between 0.0 and 100.0. At 0.0 the chemical environment in the structure is identical to the model environment and can be interpreted as a distance to a shape. In this example, a `'csm'` of 2.26 shows that the environment still shows some similarity to the model environment. Coordination environments with an `'csm'` greater than 2.5 are already considered as rather distorted. For more information on the `'csm'`, have a look at the main text. 

A more evolved strategy type, especially for intermediate coordination enviroments, is the `MultiWeightsChemenvStrategy`. In the following, the default parameters (weigths) are used. Of course, experts can also modify these weights.

In [98]:
# #Get the strategy from D. Waroquiers et al., Chem Mater., 2017, 29, 8346.
# from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import MultiWeightsChemenvStrategy
# strategy = MultiWeightsChemenvStrategy.stats_article_weights_parameters()
# # 
# lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)

In [99]:
# lse.neighbors_sets[1][0].neighb_sites_and_indices
# [6][6][0].neighb_sites_and_indices

We will start with the same oxygen site as before:

In [100]:
# #print coordination environments for a special site
# isite = 5

# print(lse.coordination_environments[isite])

This time, two coordination environments for one site exist as indicated by two appearances
of `'ce_symbol'` and there is also a `'ce_fraction'` for each of the environments. The latter indicates
the coordination environment is an intermediate between `'A:2'` (angular) and `'L:2'` (linear)
with 67% angular environment and 33% linear environment. Also, there is a `'csm'` value for both environments. As already indicated by the `'ce_fraction'`, the `'csm'` for `'A:2'` is lower (=in better agreement with the model environment) than the csm for `'L:2'`.

Now, an example follows where only one coordination environment exists. A Si occupies the corresponding site. 

In [101]:
# #another site where you have only one coordination environment (tetrahedron, T:4)
# isite = 1
# print(lse.coordination_environments[isite])

In [102]:
# import numpy as np
# np.linspace(0.5,2.0,16)

The resulting coodination environment is `'T:4'` (tetrahedron). 